In [1]:
#To process all subjects at once with MRtrix
#Have subjects in folder structured as follows: 
#study/subjects/sub1/dwi.nii
#study/subjects/sub2/dwi.nii
#...

#Run these commands from the subjects folder
import os
import nibabel as nib
import numpy as np

#Path to the subject folder. We are currently using:
#I have already preprocessed all these subjects, including applying top up and eddy correct.
#sub_path='/Users/kli9ws/Desktop/CSD_CINEPS_possible_new_subs/'
sub_path='/users/kli9ws/rds/perinatal-2/eps/CIncy_CSD_Full/subs/'

#Change directory to your diffusion folder.
os.chdir(sub_path)


In [ ]:
#Need to rename some files because MRTRIX doesn't have glob operator. Start by getting names of immediate subdirectories
subs=next(os.walk('.'))[1]

In [ ]:
#Rename for each
for sub in subs:
    try:
        os.chdir(sub_path + '/' + sub + '/dti/')
        print (sub)
        ! cp *DTI_B2000_P.nii.gz DTI_B2000_P.nii.gz
        ! cp *DTI_B2000_P.bval DTI_B2000_P.bval
        ! cp *DTI_B2000_P.bvec DTI_B2000_P.bvec
        ! cp *DTI_B2000_B0_A.nii.gz DTI_B2000_B0_A.nii.gz
    except:
        print ('Issue with' + sub)
        continue

In [ ]:
#Go back to main folder
os.chdir(sub_path)

In [ ]:
#Convert your nifty file to the .mif file MRtrix works with. Be sure to include your bvecs and bals
! foreach * : mrconvert IN/dti/DTI_B2000_P.nii.gz -fslgrad IN/dti/DTI_B2000_P.bvec IN/dti/DTI_B2000_P.bval IN/b2000.mif -force

In [ ]:
#Next, denoise your file
! foreach * : dwidenoise IN/b2000.mif IN/b2000_denoised.mif -force

In [ ]:
#Get rid of Gibbs ringing
! foreach * : mrdegibbs IN/b2000_denoised.mif IN/b2000_denoised_unringed.mif -axes 0,1

In [ ]:
#Have to use dwipreproc and not do topup and eddy the usual way. This profoundly limits options.
#MRTRIX's call to eddy can only use a pair of PA AP images. So...I will split off one of the b0s from the main image and merge it with the AP file?
! foreach * : fslsplit IN/dti/DTI_B2000_P.nii.gz IN/dti/DTI_ -t 

In [ ]:
#Have to split off just one b0 from the _AP pair too, because MRTRIX dwipreproc requires an even number
! foreach * : fslsplit IN/dti/DTI_B2000_B0_A.nii.gz IN/dti/B0_A_ -t

In [ ]:
#Get rid of gibbs ringing for the other b0s too. Couldn't hurt.
! foreach * : mrdegibbs IN/dti/DTI_0000.nii.gz IN/dti/DTI_0000_degibbs.nii.gz -axes 0,1 -force
! foreach * : mrdegibbs IN/dti/B0_A_0000.nii.gz IN/dti/B0_A_0000_degibbs.nii.gz -axes 0,1 -force

In [ ]:
#Merged the degibbs'd files
! foreach * : fslmerge -t IN/dti/combinedb0s.nii.gz IN/dti/DTI_0000_degibbs.nii.gz IN/dti/B0_A_0000_degibbs.nii.gz
#! foreach * : rm IN/dti/DTI_*.* #Not working. Again, no glob operator. Doesn't really matter, but sloppy. 

In [ ]:
#Convert the nii.gz of the combined b0s to mif 
! foreach * : mrconvert IN/dti/combinedb0s.nii.gz IN/dti/combinedb0s.mif -force

In [ ]:
#Trying dwipreproc this way
! foreach * : dwipreproc IN/b2000_denoised_unringed.mif IN/b2000_denoised_unringed_eddy.mif -pe_dir PA -rpe_pair -se_epi IN/dti/combinedb0s.mif 

In [ ]:
#Compute brain masks for each subjects. This will allow CSD to only compute fOD inside the brain
! foreach * : dwi2mask IN/b2000_denoised_unringed_eddy.mif IN/dwi_temp_mask_2000.mif 

In [ ]:
#Perform bias correction. Can't use fsl. Must use ants
! foreach * : dwibiascorrect -ants IN/b2000_denoised_unringed_eddy.mif IN/b2000_denoised_unringed_eddy_unbiased.mif -force   

In [ ]:
#Global intensity normalization across subjects required for AFD analysis. 
#For a single-tissue pipeline, can use the dwiintensitynorm script. 
#Normalisation on all subjects within a study (using a group-wise registration)
#The input and output arguments are directories containing all study images. Create directories.

! mkdir -p ../dwiintensitynorm/dwi_input
! mkdir ../dwiintensitynorm/mask_input

In [ ]:
#Need to copy each mask file into the mask_input folder, named as sub_code.mif (e.g. 4030.mif)
#Also need to copy each preprocessed dti file into the dti_input folder, named as sub_code.mif (e.g. 4030.mif)

! foreach * : cp IN/b2000_denoised_unringed_eddy_unbiased.mif ../dwiintensitynorm/dwi_input/IN.mif
! foreach * : cp IN/dwi_temp_mask_2000.mif ../dwiintensitynorm/mask_input/IN.mif

In [ ]:
#Perform global intensity normalization
#Because we are working with neonates, the standard FA threshold will only get certain populations of white matter, like corpus callosum. Need to lower threshold to approx 0.15 ?
! dwiintensitynorm ../dwiintensitynorm/dwi_input/ ../dwiintensitynorm/mask_input/ ../dwiintensitynorm/dwi_output/ ../dwiintensitynorm/fa_template.mif ../dwiintensitynorm/fa_template_wm_mask.mif -fa_threshold 0.15 -force


In [ ]:
#To output the scale factor used for all subjects 
! mrinfo ../dwiintensitynorm/dwi_output/* -property dwi_norm_scale_factor

In [ ]:
#The above command generates a dwi_output directory with normalized mif files for each subject. Put them back in the correct subject folder
#(Again, have to use cp, because symbolic linking does not work)
#Call these files 'b800_denoised_unringed_unbiased_normalized' for each subject
! foreach * : cp ../dwiintensitynorm/dwi_output/IN.mif IN/b2000_denoised_unringed_eddy_unbiased_normalized.mif
    

In [ ]:
#Compute white matter response function for each subject
! foreach * : dwi2response tournier IN/b2000_denoised_unringed_eddy_unbiased_normalized.mif IN/response.txt -force

In [ ]:
#We need one single unique response function to do spherical deconvolution on all subjects. Here we estimate the average response function
! average_response */response.txt ../group_average_response.txt    

In [ ]:
#Upsampling to an isotropic voxel size of 1.3
! foreach * : mrresize IN/b2000_denoised_unringed_eddy_unbiased_normalized.mif -vox 1.3 IN/b2000_denoised_unringed_eddy_unbiased_normalized_upsampled.mif -force 

In [ ]:
#Compute upsampled brain masks to match
! foreach * : dwi2mask IN/b2000_denoised_unringed_eddy_unbiased_normalized_upsampled.mif IN/dwi_mask_upsampled.mif -clean_scale 0 -force

In [ ]:
#Perform CSD using the average white matter response function. Here we use dwi2fod msmt_csd (even with single shell data) to benefit from the hard non-negativity constraint, which has been observed to lead to more robust outcomes.
! foreach * : dwiextract IN/b2000_denoised_unringed_eddy_unbiased_normalized_upsampled.mif - \| dwi2fod msmt_csd - ../group_average_response.txt IN/wmfod.mif -mask IN/dwi_mask_upsampled.mif -force

In [ ]:
#Going to create an unbiased FOD population template. First create directories to store all the input and output images.  
! mkdir -p ../template/fod_input
! mkdir ../template/mask_input


In [ ]:
#Copy all fod files and masks into a single folder. 
! foreach * : cp IN/wmfod.mif ../template/fod_input/PRE.mif
! foreach * : cp IN/dwi_mask_upsampled.mif ../template/mask_input/PRE.mif


In [ ]:
#Creating the population fod template. Use voxel size of 2.0 if not upsampling (native resolution)
! population_template ../template/fod_input -mask_dir ../template/mask_input ../template/wmfod_template.mif -voxel_size 1.3 -force

In [ ]:
###Testing out a new method of brain masking to make bigger masks. Failed. Some coordinate shift I can't account for. 
###Convert upsampled mif to nii
###Use bet2 to make a new brain mask
### Convert brain mask back to nii.
### Failed. Something about the conversion changes the coordinates. Might be the L/R flip I saw before between mtrix and fal

# for sub in subs:
#     try:
#         os.chdir(sub_path + '/' + sub)
#         print (sub)
#         ! mrconvert b2000_denoised_unringed_eddy_unbiased_normalized_upsampled.mif b2000_denoised_unringed_eddy_unbiased_normalized_upsampled.nii -force
#         ! bet2 b2000_denoised_unringed_eddy_unbiased_normalized_upsampled.nii b2000_denoised_unringed_eddy_unbiased_normalized_upsampled_bet.nii -m -n -f 0.2
#         ! mrconvert b2000_denoised_unringed_eddy_unbiased_normalized_upsampled_bet_mask.nii.gz dwi_mask_upsampled.mif -force
#     except:
#         print ('Issue with' + sub)
#         continue

# os.chdir(sub_path)

In [ ]:
#Register the FOD image from each subject to the FOD template:
! foreach * : mrregister IN/wmfod.mif -mask1 IN/dwi_mask_upsampled.mif ../template/wmfod_template.mif -nl_warp IN/subject2template_warp.mif IN/template2subject_warp.mif -force 

In [ ]:
#Warp all masks into template space
! foreach * : mrtransform IN/dwi_mask.mif -warp IN/subject2template_warp.mif -interp nearest -datatype bit IN/dwi_mask_in_template_space.mif -force

In [ ]:
#Compute the template mask as the intersection of all warped masks
! mrmath */dwi_mask_in_template_space.mif min ../template/template_mask.mif -datatype bit -force

In [2]:
#In this step, we segment fixels from the FOD template. The result is the fixel mask that defines the fixels for which statistical analysis will later on be performed (and hence also which fixels’ statistics can support others via the mechanism of connectivity-based fixel enhancement (CFE) [Raffelt2015]):
! fod2fixel -mask ../template/template_mask.mif -fmls_peak_value 0.10 ../template/wmfod_template.mif ../template/fixel_mask -force


fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [ERROR] Expected fixel directory ../template/fixel_mask to be empty.


In [ ]:
#Here we warp FOD images into template space without FOD reorientation
! foreach * : mrtransform IN/wmfod.mif -warp IN/subject2template_warp.mif -noreorientation IN/fod_in_template_space_NOT_REORIENTED.mif -force

In [4]:
#Here we segment each FOD lobe to identify the number and orientation of fixels in each voxel. The output also contains the apparent fibre density (AFD) value per fixel (estimated as the FOD lobe integral):
! foreach * : fod2fixel -mask ../template/template_mask.mif IN/fod_in_template_space_NOT_REORIENTED.mif IN/fixel_in_template_space_NOT_REORIENTED -afd fd.mif -force

fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [ERROR] Not a directory
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing

fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting F

fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting F

fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting FODs
fod2fixel: [WARNING] existing output files will be overwritten
fod2fixel: [100%] segmenting F

In [5]:
#Reorient the fixels
! foreach * : fixelreorient IN/fixel_in_template_space_NOT_REORIENTED IN/subject2template_warp.mif IN/fixel_in_template_space -force

fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [ERROR] Not a directory
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
f

fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixel

fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixel

fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixelreorient: [WARNING] existing output files will be overwritten
fixelreorient: [100%] reorienting fixel directions
fixel

In [6]:
#Assign subject fixels to template fixels.
#This step establishes exactly that, by matching the fixels of each individual subject to the single common set of template fixels (which then inherently also defines how they match across subjects).
! foreach * : fixelcorrespondence IN/fixel_in_template_space/fd.mif ../template/fixel_mask ../template/fd PRE.mif -force

fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [ERROR] Not a directory
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels


fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing outp

fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing outp

fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing outp

fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing output files will be overwritten
fixelcorrespondence: [100%] mapping subject fixel data to template fixels
fixelcorrespondence: [WARNING] existing outp

In [7]:
#Compute the fiber cross-section metric
! foreach * : warp2metric IN/subject2template_warp.mif -fc ../template/fixel_mask ../template/fc IN.mif -force

warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [ERROR] Not a directory
warp2metric: [ERROR] error opening image "cpfod.sh/subject2template_warp.mif"
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output file

warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
w

warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
w

warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
warp2metric: [100%] outputting warp metric(s)
warp2metric: [WARNING] existing output files will be overwritten
w

In [8]:
#However, for group statistical analysis of FC we recommend calculating the log(FC) to ensure data are centred around zero and normally distributed. Here, we create a separate fixel directory to store the log(FC) data and copy the fixel index and directions file across:
! mkdir ../template/log_fc
! cp ../template/fc/index.mif ../template/fc/directions.mif ../template/log_fc
! foreach * : mrcalc ../template/fc/IN.mif -log ../template/log_fc/IN.mif -force

mkdir: cannot create directory ‘../template/log_fc’: File exists
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [ERROR] Could not interpret string "../template/fc/cpfod.sh.mif" as either an image path or a numerical value
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_124.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_130.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_131.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_132.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_133.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_134.mif)
mrcalc: [WARNING] existing output files wil

mrcalc: [100%] computing: log (../template/fc/sub_223.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_225.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_226.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_232.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_233.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_235.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_236.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_237.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/su

mrcalc: [100%] computing: log (../template/fc/sub_517.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_518.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_521.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_522.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_523.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_524.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_526.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_529.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/su

mrcalc: [100%] computing: log (../template/fc/sub_633.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_635.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_636.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_637.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_638.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_642.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_643.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/sub_644.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: log (../template/fc/su

In [9]:
#Compute a combined measure of fibre density and cross-section (FDC)
! mkdir ../template/fdc
! cp ../template/fc/index.mif ../template/fdc
! cp ../template/fc/directions.mif ../template/fdc
! foreach * : mrcalc ../template/fd/IN.mif ../template/fc/IN.mif -mult ../template/fdc/IN.mif -force

mkdir: cannot create directory ‘../template/fdc’: File exists
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [ERROR] Could not interpret string "../template/fd/cpfod.sh.mif" as either an image path or a numerical value
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_124.mif * ../template/fc/sub_124.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_130.mif * ../template/fc/sub_130.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_131.mif * ../template/fc/sub_131.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_132.mif * ../template/fc/sub_132.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_133.mif * ../template/fc/sub_133.mif)
mrcalc: [WARNING] existing output files 

mrcalc: [100%] computing: (../template/fd/sub_204.mif * ../template/fc/sub_204.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_205.mif * ../template/fc/sub_205.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_206.mif * ../template/fc/sub_206.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_208.mif * ../template/fc/sub_208.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_209.mif * ../template/fc/sub_209.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_213.mif * ../template/fc/sub_213.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_219.mif * ../template/fc/sub_219.mif)
mrcalc: [WARNING] existing output files will be over

mrcalc: [100%] computing: (../template/fd/sub_338.mif * ../template/fc/sub_338.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_340.mif * ../template/fc/sub_340.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_341.mif * ../template/fc/sub_341.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_347.mif * ../template/fc/sub_347.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_349.mif * ../template/fc/sub_349.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_351.mif * ../template/fc/sub_351.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_352.mif * ../template/fc/sub_352.mif)
mrcalc: [WARNING] existing output files will be over

mrcalc: [100%] computing: (../template/fd/sub_580.mif * ../template/fc/sub_580.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_581.mif * ../template/fc/sub_581.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_582.mif * ../template/fc/sub_582.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_583.mif * ../template/fc/sub_583.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_586.mif * ../template/fc/sub_586.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_587.mif * ../template/fc/sub_587.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_591.mif * ../template/fc/sub_591.mif)
mrcalc: [WARNING] existing output files will be over

mrcalc: [100%] computing: (../template/fd/sub_715.mif * ../template/fc/sub_715.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_716.mif * ../template/fc/sub_716.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_718.mif * ../template/fc/sub_718.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_720.mif * ../template/fc/sub_720.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_721.mif * ../template/fc/sub_721.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_722.mif * ../template/fc/sub_722.mif)
mrcalc: [WARNING] existing output files will be overwritten
mrcalc: [100%] computing: (../template/fd/sub_723.mif * ../template/fc/sub_723.mif)
mrcalc: [WARNING] existing output files will be over

In [12]:
#Perform whole-brain fiber tractography
os.chdir('/users/kli9ws/rds/perinatal-2/eps/CIncy_CSD_Full/template')
! tckgen -angle 22.5 -maxlen 250 -minlen 10 -power 1.0 wmfod_template.mif -seed_image template_mask.mif -mask template_mask.mif -select 20000000 -cutoff 0.10 tracks_20_million.tck -force


tckgen: [WARNING] existing output files will be overwritten
tckgen: [100%] 59067551 seeds, 28112368 streamlines, 20000000 selected


In [13]:
#Perform SIFT to reduce tractography biases in the whole-brain tractogram   
! tcksift tracks_20_million.tck wmfod_template.mif tracks_2_million_sift.tck -term_number 2000000 -force

tcksift: [WARNING] existing output files will be overwritten
tcksift: [100%] Creating homogeneous processing mask
tcksift: [100%] segmenting FODs
tcksift: [100%] mapping tracks to image
tcksift:        Iteration     Removed     Remaining     Cost fn
tcksift: [done]    1709          458       2000000       31.56%
tcksift: [100%] Writing filtered tracks output file
